<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Find the distance between smoker and not

In [10]:
import cv2
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple
import math
import os

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
@dataclass
class Person:
    x1: int
    y1: int
    x2: int
    y2: int
    is_smoking: bool
    confidence: float

In [13]:
def extract_people_from_image(image_path: str) -> List[Person]:
    """
    Estrae le informazioni sulle persone dall'immagine usando i colori delle bounding box:
    - Rosso = fumatore
    - Blu = non fumatore
    """
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definiamo i range di colori per rilevare le bounding box
    # Rosso (fumatori)
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    red_mask = cv2.inRange(hsv, lower_red, upper_red)

    # Blu (non fumatori)
    lower_blue = np.array([100, 100, 100])
    upper_blue = np.array([130, 255, 255])
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Trova i contorni per ogni colore
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    people = []

    # Processa i contorni rossi (fumatori)
    for contour in red_contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20:  # Filtro per dimensione minima
            people.append(Person(
                x1=x, y1=y, x2=x+w, y2=y+h,
                is_smoking=True,
                confidence=1.0
            ))

    # Processa i contorni blu (non fumatori)
    for contour in blue_contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 20 and h > 20:  # Filtro per dimensione minima
            people.append(Person(
                x1=x, y1=y, x2=x+w, y2=y+h,
                is_smoking=False,
                confidence=1.0
            ))

    return people

In [14]:
def calculate_center_point(person: Person) -> Tuple[float, float]:
    """Calcola il punto centrale di una persona"""
    center_x = (person.x1 + person.x2) / 2
    center_y = (person.y1 + person.y2) / 2
    return (center_x, center_y)


In [15]:
def calculate_distance(p1: Person, p2: Person, pixels_per_meter: float = 100) -> float:
    """Calcola la distanza euclidea tra due persone"""
    c1 = calculate_center_point(p1)
    c2 = calculate_center_point(p2)

    pixel_distance = math.sqrt(
        (c2[0] - c1[0])**2 + (c2[1] - c1[1])**2
    )

    return pixel_distance / pixels_per_meter

In [16]:
def find_nearest_non_smoker(smoker: Person, non_smokers: List[Person]) -> Tuple[Person, float]:
    """Trova il non fumatore più vicino a un fumatore"""
    if not non_smokers:
        return None, float('inf')

    distances = [(ns, calculate_distance(smoker, ns)) for ns in non_smokers]
    return min(distances, key=lambda x: x[1])


In [17]:
def process_and_save_image(image_path: str, output_dir: str, people: List[Person]) -> None:
    """
    Processa un'immagine disegnando solo la distanza tra ogni fumatore
    e il non fumatore più vicino
    """
    os.makedirs(output_dir, exist_ok=True)
    image = cv2.imread(image_path)

    smokers = [p for p in people if p.is_smoking]
    non_smokers = [p for p in people if not p.is_smoking]

    # Disegna le bounding box originali
    for person in people:
        color = (0, 0, 255) if person.is_smoking else (255, 0, 0)  # Rosso per fumatori, Blu per non fumatori
        cv2.rectangle(image, (person.x1, person.y1), (person.x2, person.y2), color, 2)

        # Disegna il punto centrale
        center = calculate_center_point(person)
        cv2.circle(image, (int(center[0]), int(center[1])), 3, (255, 255, 0), -1)

    # Per ogni fumatore, trova il non fumatore più vicino e disegna la distanza
    for smoker in smokers:
        nearest_non_smoker, distance = find_nearest_non_smoker(smoker, non_smokers)
        if nearest_non_smoker is not None:
            s_center = calculate_center_point(smoker)
            ns_center = calculate_center_point(nearest_non_smoker)

            # Disegna la linea tra i centri
            cv2.line(image,
                    (int(s_center[0]), int(s_center[1])),
                    (int(ns_center[0]), int(ns_center[1])),
                    (255, 255, 0), 2)  # Linea più spessa per maggiore visibilità

            # Aggiungi il testo della distanza
            mid_point = ((s_center[0] + ns_center[0])//2, (s_center[1] + ns_center[1])//2)
            cv2.putText(image,
                       f"{distance:.2f}m",
                       (int(mid_point[0]), int(mid_point[1])),
                       cv2.FONT_HERSHEY_SIMPLEX,
                       0.7,  # Font più grande
                       (255, 255, 0),
                       2)  # Testo più spesso

    output_path = os.path.join(output_dir, f"processed_{os.path.basename(image_path)}")
    cv2.imwrite(output_path, image)

In [18]:
def main():
    input_dir = '/content/drive/MyDrive/test_trained_person'
    output_dir = '/content/drive/MyDrive/distance_img_process'

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            people = extract_people_from_image(image_path)
            process_and_save_image(image_path, output_dir, people)
            print(f"Processata immagine: {filename}")

if __name__ == "__main__":
    main()


Processata immagine: trained1.jpg
Processata immagine: trained2.jpg
Processata immagine: trained3.jpg
Processata immagine: trained4.jpg
Processata immagine: trained5.jpg
Processata immagine: trained6.jpg
Processata immagine: trained7.jpg
Processata immagine: trained8.jpg
Processata immagine: trained9.jpg
Processata immagine: trained10.jpg
Processata immagine: trained11.jpg
Processata immagine: trained12.jpg
Processata immagine: trained13.jpg
Processata immagine: trained14.jpg
Processata immagine: trained15.jpg
Processata immagine: trained16.jpg
Processata immagine: trained17.jpg
Processata immagine: trained18.jpg
Processata immagine: trained19.jpg
Processata immagine: trained20.jpg
Processata immagine: trained21.jpg
Processata immagine: trained22.jpg
Processata immagine: trained23.jpg
Processata immagine: trained24.jpg
Processata immagine: trained25.jpg
Processata immagine: trained26.jpg
Processata immagine: trained27.jpg
Processata immagine: trained28.jpg
Processata immagine: trained2